In [1]:
import os
import json

import pandas as pd
import requests
import psycopg2
from pymongo import MongoClient

# Get Climate data using OPEN API

In [4]:
key = "OPENAPI KEY"
start_date = 20170101
end_date = 20181231
data_size = 2*365
std_id = 159 # 부산
url17_18 = f"http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey={key}&numOfRows={data_size}&pageNo=1&dataCd=ASOS&dateCd=DAY&startDt={start_date}&endDt={end_date}&stnIds={std_id}&dataType=JSON"

In [5]:
start_date = 20190101
end_date = 20201231
data_size = 2*365
url19_20 = f"http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey={key}&numOfRows={data_size}&pageNo=1&dataCd=ASOS&dateCd=DAY&startDt={start_date}&endDt={end_date}&stnIds={std_id}&dataType=JSON"

In [6]:
start_date = 20210101
end_date = 20211231
data_size = 365
url21 = f"http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey={key}&numOfRows={data_size}&pageNo=1&dataCd=ASOS&dateCd=DAY&startDt={start_date}&endDt={end_date}&stnIds={std_id}&dataType=JSON"

In [7]:
raw_data_17_18 = requests.get(url17_18)
raw_data_19_20 = requests.get(url19_20)
raw_data_21 = requests.get(url21)

In [8]:
data_17_18 = json.loads(raw_data_17_18.text)['response']['body']['items']['item']
data_19_20 = json.loads(raw_data_19_20.text)['response']['body']['items']['item']
data_21 = json.loads(raw_data_21.text)['response']['body']['items']['item']

In [9]:
dataset = []

dataset.extend(data_17_18)
dataset.extend(data_19_20)
dataset.extend(data_21)

# Save data locally

In [10]:
# with open('dataset.txt', 'w') as f:
    # for item in dataset:
    #     f.write(f"{item}\n")
    # print("DONE")

DONE


# Read data from local directory

In [3]:
with open('dataset.txt', 'r') as f:
    dataset = f.readlines()
dataset = [data.strip().replace("'", "\"") for data in dataset]
json_data = [json.loads(data) for data in dataset]

# Upload data to Cloud (MongoDB)

In [2]:
HOST = 'MongoDB_HOST'
USER = 'MongoDB_USER'
PASSWORD = 'MongoDB_PASSWORD'
DATABASE_NAME = 'project'
COLLECTION_NAME = 'busan_climate'
MONGO_URI = f"mongodb+srv://{USER}:{PASSWORD}@{HOST}/{DATABASE_NAME}?retryWrites=true&w=majority"

In [3]:
client = MongoClient(MONGO_URI)
database = client[DATABASE_NAME]
collection = database[COLLECTION_NAME]

In [ ]:
# collection.insert_many(documents=json_data)

# Extract solar-related field, save in local database

In [4]:
datas = collection.find({}, {
    '_id': 0,
    'tm': 1,
    'sumSsHr': 1,
    'sumGsr': 1,
    'hr1MaxIcsr': 1,
    'avgTa': 1,
    'avgPa': 1,
    'avgPs': 1
})

In [46]:
# Download PostgreSQL from docker
# Create yaml file, run `docker-compose up` in terminal
conn = psycopg2.connect(
    dbname='busan_climate', 
    user='postgres', 
    password='1234',
    host='localhost'
)

cur = conn.cursor()

In [8]:
cur.execute("""DROP TABLE IF EXISTS solar""")

cur.execute("""CREATE TABLE solar(
    Id INTEGER PRIMARY KEY,
    Date TEXT,
    sumSsHr REAL,
    sumGsr REAL,
    hr1MaxIcsr REAL,
    avgTa REAL, 
    avgPa REAL,
    avgPs REAL
)""")

cur.execute("COMMIT")

In [9]:
for i, data in enumerate(datas):
    # try:
    cur.execute("""INSERT INTO solar 
        VALUES (%s, %s, 
            NULLIF(%s, '')::REAL, NULLIF(%s, '')::REAL, NULLIF(%s, '')::REAL,
            %s, %s, %s)""",
        (i, data['tm'], data['sumSsHr'], data['sumGsr'], data['hr1MaxIcsr'],
        data['avgTa'], data['avgPa'], data['avgPs']))
    # except:
    #     pass

cur.execute("COMMIT")

In [40]:
cur.execute("""DROP TABLE IF EXISTS usage""")

cur.execute("""CREATE TABLE usage (
    use_kWh INTEGER,
    pay INTEGER
)""")

cur.execute("COMMIT")

In [41]:
CSV_FILEPATH =  os.path.join(os.getcwd(), 'payment_by_use.csv')
cur.copy_expert("""COPY usage FROM STDIN DELIMITER ',' CSV HEADER""",
    open(CSV_FILEPATH, 'r'))

cur.execute("""ALTER TABLE usage ADD COLUMN reduced_pay REAL""")
cur.execute("COMMIT")

In [18]:
from Application.flask_app import payment

In [47]:
november = 275.63
month = 11

for i in range(1, 1001):
    power_fin = i - november
    pay = payment.calc_payment(month, power_fin)
    if pay <= 1150:
        pay = 1150
    else:
        pass

    cur.execute("""UPDATE usage 
                   SET reduced_pay = %s 
                   WHERE use_kwh = %s""", (pay, i))

cur.execute("COMMIT")

In [ ]:
cur.close()
conn.close()

---

# Upload to ElephantSQL for Metabase deployment by Heroku

In [ ]:
datas = collection.find({}, {
    '_id': 0,
    'tm': 1,
    'sumSsHr': 1,
    'sumGsr': 1,
    'hr1MaxIcsr': 1,
    'avgTa': 1,
    'avgPa': 1,
    'avgPs': 1
})

In [7]:
import psycopg2
from Application.flask_app import payment

host = 'ElephantSQL_HOST'
user = 'ElephantSQL_USER'
password = 'ElephantSQL_PASSWORD'
database = 'ElephantSQL_DB'

conn = psycopg2.connect(
    host=host,
    user=user,
    password=password,
    database=database
)

cur = conn.cursor()

# Copy table from Docker PostgreSQL, 
# \copy table_name to '/file/path/filename.csv' WITH DELIMITER ',' CSV HEADER

# Create solar table
cur.execute("""DROP TABLE IF EXISTS solar""")

cur.execute("""CREATE TABLE solar(
    Id INTEGER PRIMARY KEY,
    Date TEXT,
    sumSsHr REAL,
    sumGsr REAL,
    hr1MaxIcsr REAL,
    avgTa REAL, 
    avgPa REAL,
    avgPs REAL
)""")

cur.execute("COMMIT")

# Add data
CSV_FILEPATH =  os.path.join(os.getcwd(), 'solar.csv')
cur.copy_expert("""COPY solar FROM STDIN DELIMITER ',' CSV HEADER""",
    open(CSV_FILEPATH, 'r'))

cur.execute("COMMIT")


# Create usage table
cur.execute("""DROP TABLE IF EXISTS usage""")

cur.execute("""CREATE TABLE usage (
    use_kWh INTEGER,
    pay INTEGER,
    reduced_pay INTEGER
)""")

cur.execute("COMMIT")

# Create and Load usage table
CSV_FILEPATH =  os.path.join(os.getcwd(), 'usage.csv')
cur.copy_expert("""COPY usage FROM STDIN DELIMITER ',' CSV HEADER""",
    open(CSV_FILEPATH, 'r'))

cur.execute("COMMIT")

cur.close()
conn.close()